# 策略

找到30天内涨幅在80%以上的牛股，当其日线rsi回到10%以下低位时，发出买入信号

In [9]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsiday, rsi30
await init_notebook()

In [82]:
def day_features(code, name, bars, results, ft):
    """7日内出现放量涨停"""
    bars = bars[-20:]
    close = bars["close"]
    limits = [buy_limit_price(code, bar["close"], bar["frame"]) for bar in bars[:-1]]
    if np.count_nonzero(limits/close[1:] - 1 <= 1e-4) == 1:
        pos = np.argwhere(limits/close[1:] - 1 <= 1e-4)[0][0]
        if pos < 13:
            return
        
        results.append((name, code, bars[pos+1]["frame"]))

In [44]:
def m30_triggers(code, name, bars, results, ft):
    """30分钟RSI低位买入"""
    print(bars[0]["frame"])
    rsi = relative_strength_index(bars["close"])[-1]
    prsi = rsi30.get_proba(code, rsi)
    
    if (prsi and prsi < 0.1) or rsi < 10:
        results.append((name, code, bars[-1]["frame"], bars[-1]["close"], prsi, rsi))

In [109]:
async def strategy(tm=None):
    tm = arrow.get(tm) if tm else arrow.now()
    signals = await scan(day_features, 20, nstocks=-1, tm=tm)
    results = []
    for name, code, frame in signals:
        try:
            #print(name, code, frame)
            end = tf.day_shift(frame, 7)
            end = min(end, arrow.now().date())
            end = tf.combine_time(end, hour=15)
            start = tf.combine_time(frame, hour=15)
            n = tf.count_frames(start, end, FrameType.MIN30)

            bars = await get_bars(code, 18 + n, '30m', end)
            #print(bars[0]["frame"], bars[-1]["frame"], bars[-n+1]["frame"], n)

            close = bars["close"]

            bars_ = bars[-n+1:]
            rsi = relative_strength_index(close)[-n+1:]
            prsi = np.array([rsi30.get_proba(code, r) for r in rsi])

            if prsi[0] is not None:
                positions = np.argwhere(prsi < 0.11).flatten()
            else:
                positions = np.argwhere(rsi < 20).flatten()

            #print(positions)

            if len(positions):
                pos = positions[0]

                #buy_at = str(bars_[pos]["frame"])[5:16]
                buy_at = bars_[pos]["frame"]
                price = bars_[pos]["close"]

                profit = bars_[-1]["close"]/price - 1
                results.append((name, code, buy_at, price, round(100 * profit, 2)))
        except Exception as e:
            pass
            
    return pd.DataFrame(results, columns=["name", "code", "buy_at", "price", "profit"])
        
df = await strategy('2021-11-22')

2021-11-22
progress: 500/4012, elapsed: 0, ETA: 0
progress: 1000/4012, elapsed: 1, ETA: 3
progress: 1500/4012, elapsed: 1, ETA: 1
progress: 2000/4012, elapsed: 2, ETA: 2
progress: 2500/4012, elapsed: 3, ETA: 1
progress: 3000/4012, elapsed: 3, ETA: 1
progress: 3500/4012, elapsed: 4, ETA: 0
progress: 4000/4012, elapsed: 5, ETA: 0


In [110]:
df[df.buy_at<arrow.get('2021-11-25 10:00', tzinfo='Asia/Shanghai').datetime].describe()

,price,profit
count,73.000000,73.000000
mean,23.759041,2.297671
std,34.428154,6.622106
min,2.980000,-11.070000
25%,7.250000,-1.390000
50%,13.630000,1.000000
75%,25.440002,4.030000
max,247.199997,28.980000
